In [1]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict

In [2]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()

In [3]:
# already sorted by session, timestamp
df = pd.read_parquet("input/transactions.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216716096 entries, 0 to 216716095
Data columns (total 4 columns):
 #   Column   Dtype
---  ------   -----
 0   session  int32
 1   aid      int32
 2   ts       int64
 3   type     int8 
dtypes: int32(2), int64(1), int8(1)
memory usage: 3.4 GB


In [4]:
vocab: Dict[str, int] = {
    "<s>": 0,
    "<pad>": 1,
    "</s>": 2,
    "<unk>": 3,
    "<mask>": 4,
    "<cls>": 5,
    "<click>": 6,
    "<cart>": 7,
    "<order>": 8,
}
_next = len(vocab)
prev = None
words = []
rows = []
for t in tqdm(df.itertuples()):
    curr = int(getattr(t, "session"))
    if prev is not None and prev!=curr and len(words)!=0:
        rows.append({
            "session": prev,
            "seq": " ".join(words),
            "length": len(words),
        })
        words = []
    prev = curr
    _type = int(getattr(t, "type"))
    first = "<click>"
    if _type==1:
        first = "<cart>"
    elif _type==2:
        first = "<order>"
    aid = str(getattr(t, "aid"))
    if aid not in vocab:
        vocab[aid] = _next
        _next += 1
    second = aid
    words += [first, second]
if len(words)!=0:
    rows.append({
        "session": prev,
        "seq": " ".join(words),
        "length": len(words),
    })

216716096it [08:42, 414826.93it/s]


In [5]:
print(f"len(vocab)={len(vocab):,}")
with open("output/vocab.json", "w") as f:
    json.dump(vocab, f)
with open("output/vocab.txt", "w") as f:
    words = [""] * len(vocab)
    for k,v in vocab.items():
        words[v] = k
    for w in words:
        f.write(f"{w}\n")

len(vocab)=1,855,612


In [6]:
df = pd.DataFrame.from_records(rows)
cols = ["session"]
df[cols] = df[cols].astype(np.int32)
cols = ["length"]
df[cols] = df[cols].astype(np.int16)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12899779 entries, 0 to 12899778
Data columns (total 3 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   session  int32 
 1   seq      object
 2   length   int16 
dtypes: int16(1), int32(1), object(1)
memory usage: 172.2+ MB


In [7]:
df.describe(percentiles=percentiles)

,session,length
count,1.289978e+07,1.289978e+07
mean,6.449889e+06,3.359997e+01
std,3.723846e+06,6.715476e+01
min,0.000000e+00,4.000000e+00
1%,1.289978e+05,4.000000e+00
5%,6.449889e+05,4.000000e+00
10%,1.289978e+06,4.000000e+00
20%,2.579956e+06,6.000000e+00
30%,3.869933e+06,6.000000e+00
40%,5.159911e+06,8.000000e+00


In [8]:
df.head()

session  \
0        0   
1        1   
2        2   
3        3   
4        4   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [9]:
%%time
assert df.notna().all(axis=None)
df.to_parquet("output/sequences.parquet", index=False)

Wall time: 12.7 s
